### The Basics of LangChain

In this notebook we'll explore exactly what LangChain is doing - and implement a straightforward example that lets us ask questions of any Arxiv.org paper we want!

First things first, let's get our dependencies all set!

In [2]:
!pip install openai langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.5 MB/s eta 0:00:00


You'll need to have an OpenAI API key for this next part - see [this](https://www.onmsft.com/how-to/how-to-get-an-openai-api-key/) if you haven't already set one up!

In [3]:
import os 
os.environ["OPENAI_API_KEY"] = ""

#### Helper Functions (run this cell)

In [4]:
from IPython.display import display, Markdown

def disp_markdown(text: str) -> None:
  display(Markdown(text))

### Our First LangChain ChatModel



---


<div class="warn">Note: Information on OpenAI's <a href=https://openai.com/pricing>pricing</a> and <a href=https://openai.com/policies/usage-policies>usage policies.</a></div>



---



Now that we're set-up with OpenAI's API - we can begin making our first ChatModel!

There's a few important things to consider when we're using LangChain's ChatModel that are outlined [here](https://python.langchain.com/en/latest/modules/models/chat.html)

Let's begin by initializing the model with OpenAI's `gpt-3.5-turbo` (ChatGPT) model.

We're not going to be leveraging the [streaming](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html) capabilities in this Notebook - just the basics to get us started!

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

If we look at the [Chat completions](https://platform.openai.com/docs/guides/chat) documentation for OpenAI's chat models - we'll see that there are a few specific fields we'll need to concern ourselves with:

`role`
- This refers to one of three "roles" that interact with the model in specific ways.
- The `system` role is an optional role that can be used to guide the model toward a specific task. Examples of `system` messages might be: 
  - You are an expert in Python, please answer questions as though we were in a peer coding session.
  - You are the world's leading expert in stamps.

  These messages help us "prime" the model to be more aligned with our desired task!

- The `user` role represents, well, the user!
- The `assistant` role lets us act in the place of the model's outputs. We can (and will) leverage this for some few-shot prompt engineering!

Each of these roles has a class in LangChain to make it nice and easy for us to use! 

Let's look at an example.

In [ ]:
from langchain.schema import (
    AIMessage, 
    HumanMessage,
    SystemMessage
)

# The SystemMessage is associated with the system role
system_message = SystemMessage(content="You are a food critic.")

# The HumanMessage is associated with the user role
user_message = HumanMessage(content="Do you think Kraft Dinner constitues fine dining?")

# The AIMessage is associated with the assistant role
assistant_message = AIMessage(content="Egads! No, it most certainly does not!")

Now that we have those messages set-up, let's send them to `gpt-3.5-turbo` with a new user message and see how it does!

It's easy enough to do this - the ChatOpenAI model accepts a list of inputs!

In [ ]:
second_user_message = HumanMessage(content="What about Red Lobster, surely that is fine dining!")

# create the list of prompts
list_of_prompts = [
    system_message,
    user_message,
    assistant_message,
    second_user_message
]

# we can just call our chat_model on the list of prompts!
chat_model(list_of_prompts)

AIMessage(content='While Red Lobster is a popular chain restaurant known for its seafood, it is not typically considered fine dining. Fine dining typically refers to restaurants with upscale decor, high-quality ingredients, and expertly crafted dishes that are often served with a higher level of service. While Red Lobster may have some of these elements, it is more of a casual dining experience.', additional_kwargs={})

Great! That's inline with what we expected to see!

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [ ]:
# note the method `to_messages()`, that's what converts our formatted prompt into 
formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="cheeses", MOOD="quite tired", CONTENT="Hi, what are the finest cheeses?").to_messages()

disp_markdown(chat_model(formatted_chat_prompt).content)

Hello! There are many fine cheeses from around the world, and it's difficult to narrow it down to just a few. However, here are some of the finest cheeses that you might want to try:

1. Parmigiano Reggiano - an Italian hard cheese with a nutty and salty flavor.

2. Roquefort - a French blue cheese with a creamy texture and a sharp, tangy flavor.

3. Gouda - a Dutch cheese with a buttery, nutty flavor and a smooth texture.

4. Manchego - a Spanish sheep's milk cheese with a sharp, nutty flavor.

5. Brie - a French soft cheese with a creamy texture and a mild, buttery flavor.

6. Cheddar - an English cheese with a sharp, tangy flavor and a crumbly texture.

7. Emmental - a Swiss cheese with a nutty, sweet flavor and large holes.

8. Camembert - a French soft cheese with a creamy texture and a rich, earthy flavor.

9. Feta - a Greek cheese with a tangy, salty flavor and a crumbly texture.

10. Mozzarella - an Italian cheese with a mild, creamy flavor and a stretchy texture.

These are just a few examples of some of the finest cheeses in the world. There are so many more to discover and enjoy!

### Putting the Chain in LangChain

In essense, a chain is exactly as it sounds - it helps us chain actions together.

Let's take a look at an example.

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

disp_markdown(chain.run(SUBJECT="classic cars", MOOD="angry", CONTENT="Is the 67 Chevrolet Impala a good vehicle?"))

As an AI language model, I do not have emotions and therefore cannot feel angry. However, I can provide you with information about the 1967 Chevrolet Impala.

The 1967 Chevrolet Impala is considered one of the most iconic American classic cars. It was a full-size car that was available in a variety of body styles, including a convertible, coupe, and sedan. The Impala featured a powerful V8 engine and a comfortable interior, making it a popular choice for those who wanted a combination of performance and luxury.

Overall, the 1967 Chevrolet Impala is considered a good vehicle. It has a strong reputation for reliability, and its classic styling has made it a popular choice among collectors and enthusiasts alike. However, as with any classic car, maintenance and repairs can be costly, so it's important to do your research and make sure you're prepared for the investment before making a purchase.

### Index Local Files

Now that we've got our first chain running, let's talk about indexing and what we can do with it!

For the purposes of this tutorial, we'll be using the word "index" to refer to a collection of documents organized in a way that is easy for LangChain to access them as a "Retriever".

Let's check out the Retriever set-up! First, a new dependency!

In [34]:
!pip install chromadb tiktoken nltk -q

In [35]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Before we can get started with our chain - we'll have to include some kind of text that we want to include as potential context. 

Let's use Douglas Adam's [The Hitch Hiker's Guide to the Galaxy](https://erki.lap.ee/failid/raamatud/guide1.txt) as our text file. 

In [6]:
%pwd

'/content'

In [7]:
!wget https://erki.lap.ee/failid/raamatud/guide1.txt

--2023-04-26 14:48:29--  https://erki.lap.ee/failid/raamatud/guide1.txt
Resolving erki.lap.ee (erki.lap.ee)... 185.158.177.102
Connecting to erki.lap.ee (erki.lap.ee)|185.158.177.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291862 (285K) [text/plain]
Saving to: ‘guide1.txt’

guide1.txt          100%[===================>] 285.02K   665KB/s    in 0.4s    

2023-04-26 14:48:30 (665 KB/s) - ‘guide1.txt’ saved [291862/291862]



In [11]:
from langchain.document_loaders import TextLoader
loader = TextLoader('guide1.txt', encoding='utf8')

Now we can set up our first Index!

More detail can be found [here](https://python.langchain.com/en/latest/modules/indexes/getting_started.html) but we'll skip to a more functional implementation!

In [14]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Now that we have our Index set-up, we can query it straight away!

In [18]:
query = "What are the importances of towels?"
index.query_with_sources(query)

{'question': 'What are the importances of towels?',
 'answer': ' Towels are important because they have immense psychological value, can be used for warmth, protection, and distress signals, and can be used to indicate that a person is well-prepared for a journey.\n',
 'sources': 'guide1.txt'}

### Putting it All Together

Now that we have a simple idea of how we prompt, what a chain is, and what an index is - let's put it all together!

In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import NLTKTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

with open("guide1.txt") as f:
    hitchhikersguide = f.read()

text_splitter = NLTKTextSplitter()
texts = text_splitter.split_text(hitchhikersguide)

embeddings = OpenAIEmbeddings()

In [37]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

In [40]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")
query = "Why are towels important?"
docs = docsearch.get_relevant_documents(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nTowels are important because they have both practical and psychological value. In the context of the story, Arthur Dent uses a towel to protect himself from a bulldozer in his garden. Towels can also be used for warmth, as a distress signal, for hand-to-hand combat, and to ward off noxious fumes. They also have psychological value, as a strag (non-hitch hiker) will assume that a hitch hiker who has a towel with them is also in possession of other items such as a toothbrush, face flannel, soap, tin of biscuits, flask, compass, map, ball of string, gnat spray, wet weather gear, and a space suit. This gives the hitch hiker a sense of respect and admiration. Towels can also be used to protect oneself from the elements, such as when Arthur Dent used a towel to protect himself from a bulldozer in his garden. In addition, towels can be used to dry off after a swim, to mop up spills, and to provide a comfortable place to sit or lie down. Towels can also be used to protect 